In [1]:
import os
import mne
import numpy as np

In [2]:
processed_fif_file = 'subj04_session2_epo.fif'
processed_fif_path = os.path.join('..', 'eeg_data', 'final_eeg', '05_125', processed_fif_file)
epochs = mne.read_epochs(processed_fif_path)

Reading /Users/charles/Developer/all-joined/alljoined_preprocessing/snr/../eeg_data/final_eeg/05_125/subj04_session2_epo.fif ...
    Found the data of interest:
        t =     -50.78 ...     599.61 ms
        0 CTF compensation matrices available
Not setting metadata
3840 matching events found
No baseline correction applied
0 projection items activated


In [6]:
# Define the time window in seconds
tmin, tmax = 0.1, 0.3

# We will calculate SME for each unique event in the epochs
event_ids = epochs.event_id
sme_results = {}

for event_id, _ in event_ids.items():
    # Select epochs for the current event_id
    epochs_specific = epochs[event_id]
    
    # Get data and times, restrict to the time window of interest
    data = epochs_specific.get_data(picks='eeg')  # shape: (n_epochs, n_channels, n_times)
    times = epochs_specific.times
    time_window_mask = (times >= tmin) & (times <= tmax)
    
    # Calculate mean amplitude in the specified time window for each epoch
    mean_amplitudes = data[:, :, time_window_mask].mean(axis=2)  # shape: (n_epochs, n_channels)
    
    # Calculate the standard deviation across epochs for each channel
    sd = np.std(mean_amplitudes, axis=0)
    
    # Calculate N (the number of epochs for the current event_id)
    N = len(epochs_specific)
    
    # Calculate the Simple Measurement Error (SME) for each channel
    SME = sd / np.sqrt(N)
    
    # Store the results
    sme_results[event_id] = SME

# Optionally, print the SME for each event_id (and each channel, if you wish)
for event_id, SME in sme_results.items():
    print(f"Event ID: {event_id}")
    for i, sme_value in enumerate(SME):
        print(f"  Channel {i+1}: SME = {sme_value}")

Event ID: 1
  Channel 1: SME = 3.323895326309351e-06
  Channel 2: SME = 3.4939917189742096e-06
  Channel 3: SME = 2.1731515835518327e-06
  Channel 4: SME = 2.4246508473190506e-06
  Channel 5: SME = 1.750741666791121e-06
  Channel 6: SME = 1.7064645008192129e-06
  Channel 7: SME = 1.8649882931643711e-06
  Channel 8: SME = 6.602947339972084e-07
  Channel 9: SME = 1.0649869493626662e-06
  Channel 10: SME = 9.91425180531427e-07
  Channel 11: SME = 1.3031047561524597e-06
  Channel 12: SME = 1.2203158212788836e-06
  Channel 13: SME = 2.4727597069600365e-06
  Channel 14: SME = 8.030605190182017e-07
  Channel 15: SME = 1.7665822523294356e-06
  Channel 16: SME = 1.4459707897282402e-06
  Channel 17: SME = 1.4919134938031132e-06
  Channel 18: SME = 9.179909777404909e-08
  Channel 19: SME = 6.886639026379231e-07
  Channel 20: SME = 6.102093014593638e-07
  Channel 21: SME = 7.487556945368185e-07
  Channel 22: SME = 1.0746943338980457e-06
  Channel 23: SME = 1.7074902832549467e-06
  Channel 24: SME 